In [1]:
import dask.bag as db

#from nested_map import web_iter, map_wrapper

from t_analysis_flat import web_iter, map_wrapper
grid = {}
grid['b']=[0,1,2,3]
query_list = ['GMO','Genetically_Modified_Organism','Transgenic','Vaccine']
grid['search_term'] = [ (i, q) for i,q in enumerate(query_list) ]
from sklearn.grid_search import ParameterGrid
grid = list(ParameterGrid(grid))
grid = [(dicti['search_term'][0],dicti['search_term'][1],dicti['b']) for dicti in grid ]
from dask.distributed import Client
import dask.bag as db
grid = db.from_sequence(grid,npartitions = 8)
list_per_links = list(db.map(web_iter,grid).compute())
import pandas as pd
#df = pd.DataFrame(data=list_per_links)
#print(list_per_links)
#df

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno -3] Temporary failure in name resolution>


/opt/conda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


['bing_0.p', 'bing_1.p', 'bing_2.p', 'bing_3.p', 'bing_4.p', 'bing_5.p', 'bing_6.p', 'bing_7.p', 'bing_8.p', 'bing_9.p', 'bing_10.p', 'bing_11.p', 'bing_12.p', 'bing_13.p', 'bing_14.p', 'bing_15.p', 'bing_16.p', 'bing_17.p', 'bing_18.p', 'bing_19.p', 'bing_20.p', 'bing_21.p', 'bing_22.p', 'bing_23.p', 'bing_24.p', 'bing_25.p', 'bing_26.p']
['gScholar_0.p', 'gScholar_1.p', 'gScholar_2.p', 'gScholar_3.p', 'gScholar_4.p', 'gScholar_5.p', 'gScholar_6.p', 'gScholar_7.p', 'gScholar_8.p', 'gScholar_9.p', 'gScholar_10.p', 'gScholar_11.p', 'gScholar_12.p', 'gScholar_13.p', 'gScholar_14.p', 'gScholar_15.p', 'gScholar_16.p', 'gScholar_17.p', 'gScholar_18.p', 'gScholar_19.p', 'gScholar_20.p', 'gScholar_21.p', 'gScholar_22.p', 'gScholar_23.p', 'gScholar_24.p', 'gScholar_25.p', 'gScholar_26.p', 'gScholar_27.p', 'gScholar_28.p', 'gScholar_29.p', 'gScholar_30.p', 'gScholar_31.p', 'gScholar_32.p', 'gScholar_33.p', 'gScholar_34.p', 'gScholar_35.p', 'gScholar_36.p', 'gScholar_37.p', 'gScholar_38.p', 'gSc


Analyzing Search Engine 2 of 4: Link 12

Analyzing Search Engine 2 of 4: Link 13

Analyzing Search Engine 3 of 4: Link 7

Analyzing Search Engine 2 of 4: Link 13

Analyzing Search Engine 2 of 4: Link 14

Analyzing Search Engine 3 of 4: Link 8

Analyzing Search Engine 2 of 4: Link 14

Analyzing Search Engine 2 of 4: Link 15

Analyzing Search Engine 1 of 4: Link 1

Analyzing Search Engine 2 of 4: Link 15

Analyzing Search Engine 3 of 4: Link 9
Analyzing Search Engine 2 of 4: Link 16


Analyzing Search Engine 3 of 4: Link 8

Analyzing Search Engine 2 of 4: Link 16

Analyzing Search Engine 2 of 4: Link 17

Analyzing Search Engine 3 of 4: Link 10

Analyzing Search Engine 2 of 4: Link 17

Analyzing Search Engine 2 of 4: Link 18

Analyzing Search Engine 3 of 4: Link 9
Analyzing Search Engine 2 of 4: Link 18


Analyzing Search Engine 2 of 4: Link 19

Analyzing Search Engine 3 of 4: Link 11

Analyzing Search Engine 2 of 4: Link 19

Analyzing Search Engine 2 of 4: Link 20

Analyzing Search Engi


Analyzing Search Engine 3 of 4: Link 24

Analyzing Search Engine 3 of 4: Link 13

Analyzing Search Engine 1 of 4: Link 6

Analyzing Search Engine 1 of 4: Link 6

Analyzing Search Engine 1 of 4: Link 7

Analyzing Search Engine 1 of 4: Link 7

Analyzing Search Engine 3 of 4: Link 14

Analyzing Search Engine 1 of 4: Link 8

Analyzing Search Engine 1 of 4: Link 8

Analyzing Search Engine 1 of 4: Link 9

Analyzing Search Engine 3 of 4: Link 15

Analyzing Search Engine 1 of 4: Link 9

Analyzing Search Engine 1 of 4: Link 10

Analyzing Search Engine 1 of 4: Link 10

Analyzing Search Engine 3 of 4: Link 16

Analyzing Search Engine 3 of 4: Link 17

Analyzing Search Engine 1 of 4: Link 11

Analyzing Search Engine 3 of 4: Link 18

Analyzing Search Engine 1 of 4: Link 11


Analyzing Search Engine 3 of 4: Link 19
Analyzing Search Engine 3 of 4: Link 20

Analyzing Search Engine 3 of 4: Link 21

Analyzing Search Engine 3 of 4: Link 22

Analyzing Search Engine 1 of 4: Link 12

Analyzing Search Engine

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [25]:
#print(list_per_links[-11])
print(len(list_per_links))
import numpy as np
#print(np.shape(list_per_links))

remove_empty = [i for i in list_per_links if len(i)>0 ]
unravel = []
for i in remove_empty:
    unravel.extend(i)
print(len(unravel))
print(np.shape(unravel))

16
289
(289, 5)


In [32]:
#print(list_per_links[1])\

bings = [ f for f in unravel if f[0]['se'] =='bing_']
googles = [ f for f in unravel if f[0]['se'] =='google_']
GMOs = [ f for f in unravel if f[0]['keyword'] =='GMO']
#print(googles)
print(GMOs[-3])

[{'se': 'bing_', 'keyword': 'GMO', 'wcount': 328, 'sentcount': 5, 'stfreq': 0, 'sp': 0.049933862433862435, 'ss': 0.3767857142857143, 'fkg': 15.5, 'fre': 7.42, 'smog': 11.0, 'cliau': 25.85, 'ri': 20.9, 'gf': 19.679024390243903, 'dcr': 9.2, 'dw': 102, 'lwf': 9.0}, {0: 96, 1: 17, 2: 87, 3: 26, 4: 81}, {(0, 0): 4.5, (0, 1): 2.7, (0, 2): 0.9, (0, 3): 1.8, (0, 4): 0.9, (0, 5): 0.9, (0, 6): 4.5, (0, 7): 6.3, (0, 8): 2.7, (0, 9): 0.9, (0, 10): 3.6, (0, 11): 2.7, (0, 12): 0.9, (0, 13): 0.9, (0, 14): 0.9, (0, 15): 0.9, (0, 16): 1.8, (0, 17): 0.9, (0, 18): 0.9, (0, 19): 4.5, (0, 20): 3.6, (0, 21): 0.9, (0, 22): 0.9, (0, 23): 3.6, (0, 24): 0.9, (0, 25): 0.9, (0, 26): 0.9, (0, 27): 2.7, (0, 28): 0.9, (0, 29): 2.7, (0, 30): 0.9, (0, 31): 0.9, (0, 32): 0.9, (0, 33): 1.8, (0, 34): 4.5, (0, 35): 2.7, (0, 36): 0.9, (0, 37): 3.6, (0, 38): 3.6, (0, 39): 2.7, (0, 40): 0.9, (0, 41): 3.6, (0, 42): 0.9, (0, 43): 2.7, (0, 44): 1.8, (0, 45): 0.9, (0, 46): 0.9, (0, 47): 0.9, (0, 48): 0.9, (0, 49): 0.9, (0, 50): 

In [10]:

#filteredbyse = [ j[0][0] for j in remove_empty if len(j)>0 ]#if j['se']=='google_' ]
#filteredbyse = [ j[0][0] for j in remove_empty if len(j)>0 ]#if j['se']=='google_' ]
'''
bings = [ f for f in remove_empty if f['se'] =='bing_']
googles = [ f for f in remove_empty if f['se'] =='google_']
GMOs = [ f for f in remove_empty if f['keyword'] =='GMO']
'''
#print(googles)
#print(GMOs)
#print(yahoos)


[{'se': 'google_', 'keyword': 'GMO', 'wcount': 5076, 'sentcount': 188, 'stfreq': 161, 'sp': 0.055084360084360046, 'ss': 0.4824196482529815, 'fkg': 12.9, 'fre': 24.85, 'smog': 10.3, 'cliau': 24.17, 'ri': 19.2, 'gf': 17.930464933018126, 'dcr': 8.56, 'dw': 1382, 'lwf': 4.583333333333333}, {'se': 'google_', 'keyword': 'Genetically_Modified_Organism', 'wcount': 5732, 'sentcount': 213, 'stfreq': 0, 'sp': 0.03209786832802705, 'ss': 0.4672258106385091, 'fkg': 14.1, 'fre': 16.28, 'smog': 11.0, 'cliau': 26.03, 'ri': 20.8, 'gf': 17.289351011863225, 'dcr': 8.3, 'dw': 1463, 'lwf': 4.916666666666667}, {'se': 'google_', 'keyword': 'Transgenic', 'wcount': 5735, 'sentcount': 277, 'stfreq': 235, 'sp': -0.07560581543591252, 'ss': 0.5246925566343041, 'fkg': 12.6, 'fre': 25.66, 'smog': 10.4, 'cliau': 26.78, 'ri': 21.0, 'gf': 17.998116826503924, 'dcr': 8.67, 'dw': 1617, 'lwf': 5.090909090909091}, {'se': 'google_', 'keyword': 'Vaccine', 'wcount': 4824, 'sentcount': 167, 'stfreq': 141, 'sp': 0.079247946358499

In [7]:



# For all of these,
# I think it may be best if we keep text complexity on the y axes,
# mainly for ease of reading the graphs. For the correlation
# it won’t really matter which way it goes, as these both seem to be independent;
# for the others, I think bar charts would work with complexity on the
# y axes… so I figure we should stay consistent as there isn’t a real
# reason to change for #1.
# I’m having a brain fart right now… did we get
# the sentimentality analysis figured out (pro vs. con vs. neutral)?
# If not, I’m willing to work on going through and judging them one by one…
# But as for the figures, here are 3 ideas to start
# 1.       Text complexity vs site ranking
# (I think this would be really cool to see in general; do more popular sites,
#  in general, have lower complexity?), likely pooled for each major subject
# (e.g., GMO and transgenics results pooled together)

# 2.       Pro/anti/neutral vs. text complexity#

# 3.       GMO/transgenics vs. text complexity



#:
